# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [25]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [26]:
def generate_energy_df() -> pd.DataFrame:
    """
    Generate the energy dataframe.

    Returns:
        A pandas energy_df dataframe.
    """

    # load dataframe and remove header and footer
    energy_df = pd.read_excel(
        './assets/Energy Indicators.xls', 
        usecols='C:F', 
        skiprows=17,
        skipfooter=38
    )

    energy_df = energy_df.rename(columns={
        'Unnamed: 2': 'Country',
        'Petajoules': 'Energy Supply',
        'Gigajoules': 'Energy Supply per Capita',
        '%': '% Renewable'
    })

    # transform data
    energy_df = energy_df.replace({'...': np.nan})
    energy_df['Energy Supply'] = energy_df['Energy Supply'] * 1000000

    # transform Country column
    energy_df["Country"] = energy_df["Country"].\
                        replace(r"(\s\(.+\))|[0-9]+", "", regex=True)

    energy_df['Country'] = energy_df['Country'].replace({
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    })
    
    return energy_df

In [27]:
def get_GDP_df() -> pd.DataFrame:
    """
    Generate the GDP dataframe.

    Returns:
        The pandas World bank dataframe.
    """
    # YOUR CODE HERE load data, remove header, adjust column names
    gdp_df = pd.read_csv("assets/world_bank.csv", header=4)
    gdp_df = gdp_df.rename(columns={"Country Name": "Country"})

    # YOUR CODE HERE transform Country column 
    gdp_df['Country'] = gdp_df['Country'].replace({
        "Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    })

    gdp_df = gdp_df.drop(gdp_df.columns[1:-10], axis=1)

    return gdp_df

In [28]:
def get_scimago_df() -> pd.DataFrame:
    """
    Generate the Scimago dataframe.

    Returns:
        The pandas Scimago dataframe.
    """
    scimago_df = pd.read_excel("assets/scimagojr-3.xlsx")
    return scimago_df

In [29]:
def answer_one() -> pd.DataFrame:
    """
    Merge the energy, gdc and scimag dataframes into a single dataframe.

    Returns:
        The merged dataframe.
    """
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    # merge dataframes
    df = pd.merge(energy_df, gdp_df, on='Country', how='inner')
    df = pd.merge(scimago_df.iloc[:15], df, on='Country', how='inner')
    df.set_index('Country', inplace=True)
    df.sort_values('Rank', inplace=True)

    return df

In [30]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"
answer_one().head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


In [31]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [32]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [33]:
def answer_two() -> int:
    """
    How many entries are missing from the merged dataframe?

    Returns:
        The number of missing entries in the merged dataframe.
    """

    merged_df = answer_one()
    complete = pd.merge(generate_energy_df(), get_GDP_df(), on='Country', how='outer')
    complete = pd.merge(get_scimago_df(), complete, on='Country', how='outer')

    return len(complete) - len(merged_df)

In [34]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"
print(f'{answer_two()} missing entries')

304 missing entries


In [35]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [36]:
def answer_three() -> pd.Series:
    """
    What are the top 15 countries for average GDP over the last 10 years?
    
    Returns:
        A pandas Series with the average GDP over the last 10 years for 
        the top 15 countries.    
    """
    top15_df = answer_one()
    avg_gdp = top15_df.iloc[:, 10:].mean(axis=1).\
              sort_values(ascending=False)

    return avg_gdp

In [37]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"
answer_three().head(15)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [38]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [39]:
def answer_four() -> float:
    """
    By how much had the GDP changed over the 10 year span for the country 
    with the 6th largest average GDP?

    Returns:
        The GDP change for the country with the 6th largest average GDP.
    """
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()

    energy_merged = pd.merge(energy_df, gdp_df, on='Country', how='inner')
    energy_merged = energy_merged.set_index('Country')
    energy_merged['avgGDP'] = energy_merged.iloc[:, 10:].mean(axis=1)
    energy_merged = energy_merged.sort_values('avgGDP', ascending=False)

    return float(energy_merged['2015'][5] - energy_merged['2006'][5])


In [40]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"
print(f'GDP change: {answer_four()}')

GDP change: 246702696075.3999


In [41]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [42]:
def answer_five() -> float:
    """
    Calculate the mean Energy Supply per Capita.

    Returns:
        The mean Energy Supply per Capita.
    """
    energy_df = generate_energy_df()

    return float(np.nanmean(energy_df['Energy Supply per Capita']))

In [43]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"
print(f'Mean Energy Supply per Capita: {answer_five()}')

Mean Energy Supply per Capita: 90.66666666666667


In [44]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [45]:
def answer_six() -> tuple:
    """
    Get the country with maximum % Renewable and is percentage

    Returns:
        A tuple with the country that has the maximum % Renewable 
        and the percentage.
    """
    top15_df = answer_one()
    max_renewable = top15_df['% Renewable'].idxmax()
    renewable_value = top15_df['% Renewable'].max()
    return (max_renewable, float(renewable_value))

In [46]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"
print(f'{answer_six()[0]} with {answer_six()[1]}% renewable')

Brazil with 69.64803% renewable


In [47]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [48]:
def answer_seven() -> tuple:
    """
    Calculate the ratio of Self-Citations to Total Citations.

    Returns:
        A tuple with the country that has the maximum ratio and the ratio value.
    """
    top15_df = answer_one()
    top15_df['Ratio'] = top15_df['Self-citations'] / top15_df['Citations']
    max_ratio_country = top15_df['Ratio'].idxmax()
    max_ratio_value = top15_df['Ratio'].max()

    return (max_ratio_country, float(max_ratio_value))

In [49]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"
print(f'{answer_seven()[0]} with {answer_seven()[1]} citation ratio')

China with 0.6893126179389422 citation ratio


In [50]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [51]:
def answer_eight() -> str:
    """
    Calculate the third most populous country.

    Returns:
        The name of the third most populous country.
    """
    energy_df = generate_energy_df()
    energy_df['Population'] = energy_df['Energy Supply'] / energy_df['Energy Supply per Capita']
    energy_df = energy_df.sort_values('Population', ascending=False)
    
    return energy_df.iloc[2].Country

In [52]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"
print(f'{answer_eight()} has the 3rd largest population')

United States has the 3rd largest population


In [53]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [54]:
def answer_nine() -> float:
    """
    Calculate the correlation between the number of citable documents per capita and 
    the energy supply per capita.

    Returns:
        The correlation value.
    """
    top15_df = answer_one()
    top15_df['Population'] = top15_df['Energy Supply'] / top15_df['Energy Supply per Capita']
    top15_df['Citable docs per Capita'] = top15_df['Citable documents'] / top15_df['Population']

    corr = top15_df['Citable docs per Capita'].corr(top15_df['Energy Supply per Capita'])
    return corr


In [55]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"
print(f'Correlation: {answer_nine()}')

Correlation: 0.7940010435442942


In [56]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [57]:
def answer_ten() -> pd.Series:
    """
    Create a new column with a 1 if the country's % Renewable value is at or above 
    the median for all countries in the top 15, and a 0 if the country's % Renewable 
    value is below the median.

    Returns:
        A series of the new binary column.
    """
    top15_df = answer_one()
    median = top15_df['% Renewable'].median()
    top15_df['high_renew'] = top15_df['% Renewable'].apply(lambda x: 1 if x >= median else 0)

    return top15_df['high_renew']



In [58]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"
answer_ten()


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: high_renew, dtype: int64

In [59]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [60]:
def answer_eleven() -> pd.DataFrame:
    """
    Group the countries by continent and display the size, sum, mean, and std deviation for 
    the estimated population.
     
    Returns:
           The DataFrame with the aggregated values.
    """
    
    Continents  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'}
    
    energy_df = generate_energy_df()
    energy_df['Population'] = energy_df['Energy Supply'] / energy_df['Energy Supply per Capita']
    energy_df['Continent'] = energy_df['Country'].map(Continents)
    energy_df = energy_df[energy_df['Continent'].notnull()]
    
    energy_df = energy_df.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])
    
    return energy_df

In [61]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"
answer_eleven().head()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [62]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [63]:
def answer_twelve() -> pd.Series:
    """
    Cut % Renewable into 5 bins. Group the top 15 countries by Continent and % Renewable.
    
    Returns:
        The number of countries in each group.
    """
    Continents  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'}
    
    top15_df = answer_one()
    top15_df = top15_df.reset_index()
    top15_df['Continent'] = top15_df['Country'].map(Continents)
    
    top15_df['% Renewable'] = pd.cut(top15_df['% Renewable'], 5)
    top15_df = top15_df.groupby(['Continent', '% Renewable']).agg({'Country': 'count'})
    
    return top15_df[top15_df['Country'] > 0]['Country']

    

In [64]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"
print("len:", len(answer_twelve()))
answer_twelve().head(25)


len: 9


Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
Name: Country, dtype: int64

In [65]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [85]:
def answer_thirteen() -> pd.Series:
    """
    Convert the Population Estimate series to a string with thousands separator.
    
    Returns:
        The formatted population estimate series.
    """
    def format_convertor(num: int) -> str:
        """
        Convert number to string with thousands separator.
        
        Returns:
            The formatted number.
        
        """
        return '{:,}'.format(num).replace(",","x").replace(".",",").replace("x",".")
    
    top15_df = answer_one()
    top15_df['pop_est'] = top15_df['Energy Supply'] / top15_df['Energy Supply per Capita']
    top15_df['pop_est'] = top15_df['pop_est'].apply(format_convertor)
    
    return top15_df['pop_est']
    

In [87]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"
answer_thirteen().head()


Country
China                 1.367.645.161,2903225
United States          317.615.384,61538464
Japan                  127.409.395,97315437
United Kingdom         63.870.967,741935484
Russian Federation            143.500.000,0
Name: pop_est, dtype: object

In [68]:
# space for proffesor tests